### Read in Dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("C:\\Users\\Ronald\\Documents\\EECS_738\\Projects\\Says One Neuron to Another\\zoo.csv")
df.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


### Create for activation functions for network

In [2]:
def relu(x):
    return np.maximum(0,x)

def softmax(x):
    e_x = np.exp(x)
    softmax = e_x / e_x.sum(axis=1, keepdims=True)
    return softmax

### Define backpropagation for network

In [3]:
def cross_entropy_back_prop(inputs, outputs, hw, ow, hb, ob, learning_rate, ah, y):
 
    # softmax gradient
    softmax_der = (y - outputs) / outputs.shape[0]
    
    # output layer weights and biases gradients
    ow_der = np.dot(ah.T, softmax_der)
    ob_der = np.sum(softmax_der, axis = 0, keepdims = True)
    
    # hidden layer weights and biases gradients
    ah_der = np.dot(softmax_der, ow.T)
    ah_der[ah <= 0] = 0
    hw_der = np.dot(inputs.T, ah_der)
    hb_der = np.sum(ah_der, axis = 0, keepdims = True)
    
    # update output layer and hidden layer weights and biases
    ow -= learning_rate * ow_der
    hw -= learning_rate * hw_der
    
    ob -= learning_rate * ob_der 
    hb -= learning_rate * hb_der

### Define model training for network

In [4]:
def mlp(inputs, outputs, hidden_layer_width, learning_rate, epochs):
    
    d = dict()
    
    # hyperparameters
    np.random.seed(42)

    # weights for hidden and output layers
    hw = np.random.rand(len(inputs[0]), hidden_layer_width)

    ow = np.random.rand(hidden_layer_width, 7)

    # intialize biases for hidden and output layers to zeros
    hb = np.zeros((1, hidden_layer_width))
    ob = np.zeros((1, 7))

    for epoch in range(1, epochs + 1):
        # feed forward
        # hidden layer
        zh = np.dot(inputs, hw) + hb
        ah = relu(zh)

        # output layer
        zo = np.dot(ah, ow) + ob
        y = softmax(zo)

        # calculate cross entropy 
        indices = np.argmax(outputs, axis = 1).astype(int)
        predicted_probability = y[np.arange(len(y)), indices]
        log_preds = np.log(predicted_probability)
        cross_entropy = -np.sum(log_preds) / len(log_preds)        
        
        # backpropagation
        cross_entropy_back_prop(inputs, outputs, hw, ow, hb, ob, learning_rate, ah, y)
    
        if epoch == epochs:
            print("After %d epochs, cross entropy is now: %f" % (epoch, cross_entropy))
            d['hidden layer weights'] = hw
            d['output layer weights'] = ow
            d['hidden layer biases'] = hb
            d['output layer biases'] = ob
    
    return d

### Define model accuracy for network

In [5]:
def model_accuracy(test_inputs, test_outputs, hidden_weights, output_weights, hidden_biases, output_biases):
    
    # hidden layer
    zh = np.dot(test_inputs, hidden_weights) + hidden_biases
    ah = relu(zh)
    
    # output layer
    zo = np.dot(ah, output_weights) + output_biases
    y = softmax(zo)
    
    preds_correct_boolean =  np.argmax(y, 1) == np.argmax(test_outputs, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = correct_predictions / len(test_outputs)
    print("Accuracy is: %f" % accuracy)

### Shape inputs and outputs for network

In [6]:
# inputs and outputs
inputs = df.drop(columns=['animal_name', 'class_type'])
outputs = pd.get_dummies(df['class_type'].values)
outputs = outputs.values

# 80-20 split for training and testing sets
split = round(.8 * len(inputs))

X_train = np.array(inputs[:split])
X_test = np.array(inputs[split:])

y_train = np.array(outputs[:split])
y_test = np.array(outputs[split:])

### Create model

In [7]:
model = mlp(inputs=X_train, outputs=y_train, hidden_layer_width=20, learning_rate=0.05, epochs=10000)

After 10000 epochs, cross entropy is now: 0.000925


### Test model accuracy

In [8]:
hw = model['hidden layer weights']
ow = model['output layer weights']
hb = model['hidden layer biases']
ob = model['output layer biases']

model_accuracy(test_inputs=X_test, test_outputs=y_test, hidden_weights=hw, output_weights=ow, 
               hidden_biases=hb, output_biases=ob)

Accuracy is: 0.800000
